In [1]:
import pandas as pd

In [39]:
df = pd.read_csv('Karpov.Courses. Monetisation - data.csv')

In [40]:
# Подготовим два списка: один для колонок с 'revenue_', другой для 'cnt_'
revenue_columns = [col for col in df.columns if 'revenue_' in col]
cnt_columns = [col for col in df.columns if 'cnt_' in col]

# Определим изначальные колонки
base_columns = ['execution_date', 'scale', 'region']

# Развернем столбцы с 'revenue_' и 'cnt_'
df_revenue = pd.melt(df, id_vars=base_columns, value_vars=revenue_columns, 
                     var_name='service', value_name='revenue_per_service')
df_revenue['service'] = df_revenue['service'].str.replace('revenue_', '')
df_revenue = df_revenue[['execution_date', 'scale', 'region', 'service', 'revenue_per_service']]


df_cnt = pd.melt(df, id_vars=base_columns, value_vars=cnt_columns, 
                 var_name='service', value_name='cnt_per_service')
df_cnt['service'] = df_cnt['service'].str.replace('cnt_', '')
df_cnt = df_cnt[['execution_date', 'scale', 'region', 'service', 'cnt_per_service']]




In [41]:
# Объединим полученные датафреймы
df_per_service = pd.merge(df_revenue, df_cnt, on=base_columns+['service'])

In [22]:
df_per_service.to_csv('data_per_service.csv', index=False)

In [42]:
columns_clear = list(set(df.columns) - set(revenue_columns) - set(cnt_columns))

df_clear = df[columns_clear]

print(df_clear)

       scale  avg_contacts_free_ad  free_lister    revenue  free_adverts  \
0      MONTH              1.833427       7798.0   494591.0       20387.0   
1       WEEK              1.835393       3132.0    43876.0        5059.0   
2        DAY              1.824526       7446.0    64585.0       40641.0   
3      MONTH              1.871197      11149.0   629760.0       29957.0   
4      MONTH              1.847213       9176.0  1168539.0       26104.0   
...      ...                   ...          ...        ...           ...   
18034    DAY              2.060089       5308.0    58486.0       11693.0   
18035    DAY              1.976965       3461.0    46642.0        9406.0   
18036    DAY              1.942373       2274.0    22736.0        3114.0   
18037    DAY              1.723164       2706.0    43923.0        5985.0   
18038    DAY              1.848868     155831.0  3145564.0      873472.0   

       paying_lister_cnt  paying_adverts_cnt  payments_cnt  \
0                 2291.0 

In [27]:
df_clear.to_csv('df_clear.csv', index=False)

In [44]:
df_clear['execution_date'] = pd.to_datetime(df_clear['execution_date'])

# Сначала создайте копию вашего DataFrame и сдвиньте 'execution_date' на один год назад
df_prev_year = df_clear.copy()
df_prev_year['execution_date'] = df_prev_year['execution_date'] + pd.DateOffset(years=1)

# Оставьте только столбцы, необходимые для объединения и создания 'revenue_plan'
df_prev_year = df_prev_year[['execution_date', 'scale', 'revenue','region']]

# Переименуйте столбец 'revenue' на 'revenue_plan' в этой копии
df_prev_year.rename(columns={'revenue': 'revenue_plan'}, inplace=True)

# Объедините исходный DataFrame с df_prev_year по 'execution_date' и 'scale'
df_final = pd.merge(df_clear, df_prev_year, on=['execution_date', 'scale','region'], how='left')

df_final['revenue_plan'] = df_final['revenue_plan']*1.15

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
df_final = df_final.drop_duplicates()

print(df_final)

       scale  avg_contacts_free_ad  free_lister    revenue  free_adverts  \
0      MONTH              1.833427       7798.0   494591.0       20387.0   
1       WEEK              1.835393       3132.0    43876.0        5059.0   
2        DAY              1.824526       7446.0    64585.0       40641.0   
3      MONTH              1.871197      11149.0   629760.0       29957.0   
4      MONTH              1.847213       9176.0  1168539.0       26104.0   
...      ...                   ...          ...        ...           ...   
18034    DAY              2.060089       5308.0    58486.0       11693.0   
18035    DAY              1.976965       3461.0    46642.0        9406.0   
18036    DAY              1.942373       2274.0    22736.0        3114.0   
18037    DAY              1.723164       2706.0    43923.0        5985.0   
18038    DAY              1.848868     155831.0  3145564.0      873472.0   

       paying_lister_cnt  paying_adverts_cnt  payments_cnt  \
0                 2291.0 

In [46]:
df_final['deviation'] = (df_final['revenue'] - df_final['revenue_plan']) / df_final['revenue_plan'] * 100

df_final['deviation'] = df_final['deviation'].fillna(0)

In [47]:
print(df_final)

       scale  avg_contacts_free_ad  free_lister    revenue  free_adverts  \
0      MONTH              1.833427       7798.0   494591.0       20387.0   
1       WEEK              1.835393       3132.0    43876.0        5059.0   
2        DAY              1.824526       7446.0    64585.0       40641.0   
3      MONTH              1.871197      11149.0   629760.0       29957.0   
4      MONTH              1.847213       9176.0  1168539.0       26104.0   
...      ...                   ...          ...        ...           ...   
18034    DAY              2.060089       5308.0    58486.0       11693.0   
18035    DAY              1.976965       3461.0    46642.0        9406.0   
18036    DAY              1.942373       2274.0    22736.0        3114.0   
18037    DAY              1.723164       2706.0    43923.0        5985.0   
18038    DAY              1.848868     155831.0  3145564.0      873472.0   

       paying_lister_cnt  paying_adverts_cnt  payments_cnt  \
0                 2291.0 

In [48]:
df_final.to_csv('df_clear.csv', index=False)